In [1]:
# importing liabraries.
import pandas as pd
import numpy as np
import os
import PyPDF2
from PyPDF2 import PdfFileReader, PdfFileWriter


import re # regex for text pattern matching....
import string # string liabrary for getting punctuations and special charecters and ascii letters

whitelist = string.ascii_letters + string.digits + ' '

## Data Preparation

In [22]:
# naming the data files
# def Naming_the_files(input_folder):
#     for i in os.listdir(input_folder):
#         req_path = input_folder+"/"+i
#         #print(req_path)
#         if req_path.endswith("pdf"):
#             new_name = req_path.split(".")[0]+"_"+req_path.split("/")[1]+".pdf".replace("-"," ")
#             if not os.path.exists(req_path,req_path.split(".")[0]+"_"+req_path.split("/")[1]+".pdf"):
#                 os.rename(req_path,new_name)
        
# RawData/Agreements/some.pdf
# pdf splitter...

def pdf_splitter(input_file):
    inputpdf = PdfFileReader(open(input_file, "rb")) 
    for i in range(inputpdf.numPages):
        file_name =  input_file.split(".")[0]+'@'+input_file.split("/")[1]+'@'+ '-page-{}.pdf'.format(i)
        #print(file_name)
        output = PdfFileWriter()
        output.addPage(inputpdf.getPage(i))
        with open('splitted_pdfs/'+file_name.split("/")[-1], "wb") as outputStream:
            output.write(outputStream)



# converting a pdf to txt
def pdftotxt(file_path,out_path):
    #print(file_path.endswith("pdf") and file_path.split(".")[0].split("-")[-1])
    if  file_path.endswith("pdf") and int(file_path.split(".")[0].split("-")[-1]) in [0,1,2,3,4,5,6,7,8,9]:
        #print("xyz")
        pdfFileObj =open(file_path,'rb')
        pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
        #print(pdfReader.numPages) 
        number_of_pages = pdfReader.getNumPages()
        page_content = ' '
        for i in range(number_of_pages):
            pageObj = pdfReader.getPage(i)
            page_content= page_content + pageObj.extractText() # extracting the text and storing in a variable...
            #text cleaning
            page_content = page_content.lower() # transforming all letters of the text to lower case.
            page_content = page_content.replace("\n"," ") # replacing \n with space.
            page_content =  re.sub(r'\b[0-9]+\b\s*', '', page_content) # \b means word boundaries, [0,9] means all between 0 and 9, + means one or more, \s means white spaces will be substituted with empty strings.
            
            page_content =  re.sub(r'[^\x00-\x7F]+', '', page_content) # replace special charecters with empty string
            page_content =  page_content.replace(string.punctuation, ' ') # replace punctuations with space.
            page_content =  re.sub(r'\s+', ' ', page_content) # if multiple white spaces are there replace with the empty string.
            page_content = ''.join([i for i in page_content if i in whitelist]) #remove symbols
            page_content =  re.sub(' +', ' ',page_content) # replace multiple spaces with single space (For ensuring)
            page_content = page_content.strip()
            
            
        out_file = out_path+"/"+file_path.split(".")[0].split("/")[-1]+".txt"
        with open(out_file, 'a',encoding="utf-8") as text:
            text.writelines(page_content)
                            

In [23]:
# def Driving_function():
# Creating required Directories...

input_folder = 'RawData'
splitted_folder ='splitted_pdfs'
text_out_path = "txt_folder"

os.path.exists(splitted_folder) or os.makedirs(splitted_folder)
os.path.exists(text_out_path) or os.makedirs(text_out_path)

# creating an empty Data Frame, to append with data later on in the process...
Data_Frame = pd.DataFrame([],columns=['File_name','text','target'])
row_data= [] # empty list for storing each row of data so that we will append the entire list to the data farame...

# To Split the pdfs into single pages and store them in the splitted_pdfs folder...
for i in os.listdir(input_folder): # looping through RawData folder for class names...
    input_path = input_folder+"/"+i
    target= i
    #print(input_path)
    #naming the documents
    #Naming_the_files(input_folder=input_path)

    # splitting the pdfs
    for j in os.listdir(input_path): #looping through each class and getting file names as j
        splitting_file_path = input_path+"/"+j
        if j.endswith("pdf"):

            # function for splitting pdfs
            pdf_splitter(input_file=splitting_file_path)

# converting pdf to txt... and cleaning the text
for k in os.listdir(splitted_folder): # looping thorugh splitted pdfs folder and extracting files names to extract text from.
    pdf_2txt_path = splitted_folder+"/"+k
    print(pdf_2txt_path)

    # functon for extracting text from pdfs...
    pdftotxt(file_path=pdf_2txt_path,out_path=text_out_path)


# extracting text from the txt file and creating a Data Frame.
for l in os.listdir(text_out_path): # looping through txt folder ans extracting text, target, file name to create a Data Frame.
    if l.endswith("txt"):
        text_file = text_out_path+"/"+l
        with open(text_file,"r",encoding="utf-8") as f:
            txt = f.read()


        text = txt
        target= text_file.split("@")[1]
        file_name = text_file

        result = {"File_name":file_name,'text':text,"target":target}

        row_data.append(result)

        #result = getting_text(row_data=row_data,text_file=text_file)


Data_Frame = Data_Frame.append(row_data,ignore_index=True)
Data_Frame.to_excel("Data_Prep.xlsx")


splitted_pdfs/0803_job_description_vol_coordinator_0053@Human Resources@-page-0.pdf
splitted_pdfs/0803_job_description_vol_coordinator_0053@Human Resources@-page-1.pdf
splitted_pdfs/11@Human Resources@-page-0.pdf
splitted_pdfs/11@Human Resources@-page-1.pdf
splitted_pdfs/11@Human Resources@-page-2.pdf
splitted_pdfs/11@Human Resources@-page-3.pdf
splitted_pdfs/2015springvaluationreport@Valuations@-page-0.pdf
splitted_pdfs/2015springvaluationreport@Valuations@-page-1.pdf
splitted_pdfs/2015springvaluationreport@Valuations@-page-2.pdf
splitted_pdfs/2015springvaluationreport@Valuations@-page-3.pdf
splitted_pdfs/2016@Valuations@-page-0.pdf
splitted_pdfs/2016@Valuations@-page-1.pdf
splitted_pdfs/2016@Valuations@-page-10.pdf
splitted_pdfs/2016@Valuations@-page-11.pdf
splitted_pdfs/2016@Valuations@-page-12.pdf
splitted_pdfs/2016@Valuations@-page-13.pdf
splitted_pdfs/2016@Valuations@-page-14.pdf
splitted_pdfs/2016@Valuations@-page-15.pdf
splitted_pdfs/2016@Valuations@-page-16.pdf
splitted_pdfs/2

splitted_pdfs/aramark-tax-2013-05-19@Taxes@-page-0.pdf
splitted_pdfs/aramark-tax-2013-05-19@Taxes@-page-1.pdf
splitted_pdfs/aramark-tax-2013-05-19@Taxes@-page-10.pdf
splitted_pdfs/aramark-tax-2013-05-19@Taxes@-page-2.pdf
splitted_pdfs/aramark-tax-2013-05-19@Taxes@-page-3.pdf
splitted_pdfs/aramark-tax-2013-05-19@Taxes@-page-4.pdf
splitted_pdfs/aramark-tax-2013-05-19@Taxes@-page-5.pdf
splitted_pdfs/aramark-tax-2013-05-19@Taxes@-page-6.pdf
splitted_pdfs/aramark-tax-2013-05-19@Taxes@-page-7.pdf
splitted_pdfs/aramark-tax-2013-05-19@Taxes@-page-8.pdf
splitted_pdfs/aramark-tax-2013-05-19@Taxes@-page-9.pdf
splitted_pdfs/AxiomValuationSampleCertifiedReport@Valuations@-page-0.pdf
splitted_pdfs/AxiomValuationSampleCertifiedReport@Valuations@-page-1.pdf
splitted_pdfs/AxiomValuationSampleCertifiedReport@Valuations@-page-10.pdf
splitted_pdfs/AxiomValuationSampleCertifiedReport@Valuations@-page-11.pdf
splitted_pdfs/AxiomValuationSampleCertifiedReport@Valuations@-page-12.pdf
splitted_pdfs/AxiomValuati

splitted_pdfs/blackstone-tax-2007@Taxes@-page-6.pdf
splitted_pdfs/blackstone-tax-2007@Taxes@-page-7.pdf
splitted_pdfs/blackstone-tax-2007@Taxes@-page-8.pdf
splitted_pdfs/blackstone-tax-2007@Taxes@-page-9.pdf
splitted_pdfs/Board_Policy_Manual_Examples@Human Resources@-page-0.pdf
splitted_pdfs/Board_Policy_Manual_Examples@Human Resources@-page-1.pdf
splitted_pdfs/Board_Policy_Manual_Examples@Human Resources@-page-10.pdf
splitted_pdfs/Board_Policy_Manual_Examples@Human Resources@-page-100.pdf
splitted_pdfs/Board_Policy_Manual_Examples@Human Resources@-page-101.pdf
splitted_pdfs/Board_Policy_Manual_Examples@Human Resources@-page-102.pdf
splitted_pdfs/Board_Policy_Manual_Examples@Human Resources@-page-103.pdf
splitted_pdfs/Board_Policy_Manual_Examples@Human Resources@-page-104.pdf
splitted_pdfs/Board_Policy_Manual_Examples@Human Resources@-page-105.pdf
splitted_pdfs/Board_Policy_Manual_Examples@Human Resources@-page-106.pdf
splitted_pdfs/Board_Policy_Manual_Examples@Human Resources@-page-10

splitted_pdfs/clear-channel-tax-2005-12-21@Taxes@-page-10.pdf
splitted_pdfs/clear-channel-tax-2005-12-21@Taxes@-page-11.pdf
splitted_pdfs/clear-channel-tax-2005-12-21@Taxes@-page-12.pdf
splitted_pdfs/clear-channel-tax-2005-12-21@Taxes@-page-13.pdf
splitted_pdfs/clear-channel-tax-2005-12-21@Taxes@-page-14.pdf
splitted_pdfs/clear-channel-tax-2005-12-21@Taxes@-page-15.pdf
splitted_pdfs/clear-channel-tax-2005-12-21@Taxes@-page-16.pdf
splitted_pdfs/clear-channel-tax-2005-12-21@Taxes@-page-17.pdf
splitted_pdfs/clear-channel-tax-2005-12-21@Taxes@-page-18.pdf
splitted_pdfs/clear-channel-tax-2005-12-21@Taxes@-page-19.pdf
splitted_pdfs/clear-channel-tax-2005-12-21@Taxes@-page-2.pdf
splitted_pdfs/clear-channel-tax-2005-12-21@Taxes@-page-20.pdf
splitted_pdfs/clear-channel-tax-2005-12-21@Taxes@-page-21.pdf
splitted_pdfs/clear-channel-tax-2005-12-21@Taxes@-page-22.pdf
splitted_pdfs/clear-channel-tax-2005-12-21@Taxes@-page-23.pdf
splitted_pdfs/clear-channel-tax-2005-12-21@Taxes@-page-24.pdf
splitted_

splitted_pdfs/DeedOfTrustShortForm (2)@Deeds@-page-3.pdf
splitted_pdfs/DeedOfTrustShortForm (2)@Deeds@-page-4.pdf
splitted_pdfs/DeedOfTrustShortForm (2)@Deeds@-page-5.pdf
splitted_pdfs/DeedOfTrustShortForm (2)@Deeds@-page-6.pdf
splitted_pdfs/DeedOfTrustShortForm (2)@Deeds@-page-7.pdf
splitted_pdfs/DeedOfTrustShortForm (2)@Deeds@-page-8.pdf
splitted_pdfs/DeedOfTrustShortForm@Deeds@-page-0.pdf
splitted_pdfs/DeedOfTrustShortForm@Deeds@-page-1.pdf
splitted_pdfs/DeedOfTrustShortForm@Deeds@-page-2.pdf
splitted_pdfs/DeedOfTrustShortForm@Deeds@-page-3.pdf
splitted_pdfs/DeedOfTrustShortForm@Deeds@-page-4.pdf
splitted_pdfs/DeedOfTrustShortForm@Deeds@-page-5.pdf
splitted_pdfs/DeedOfTrustShortForm@Deeds@-page-6.pdf
splitted_pdfs/DeedOfTrustShortForm@Deeds@-page-7.pdf
splitted_pdfs/DeedOfTrustShortForm@Deeds@-page-8.pdf
splitted_pdfs/Deed_of_Trust_-_RALLimited_Trust@Deeds@-page-0.pdf
splitted_pdfs/Deed_of_Trust_-_RALLimited_Trust@Deeds@-page-1.pdf
splitted_pdfs/Deed_of_Trust_-_RALLimited_Trust@Deed

...

splitted_pdfs/pence-distribution-1996@Taxes@-page-3.pdf
splitted_pdfs/pence-distribution-1996@Taxes@-page-4.pdf
splitted_pdfs/pence-distribution-1996@Taxes@-page-5.pdf
splitted_pdfs/pence-distribution-1996@Taxes@-page-6.pdf
splitted_pdfs/pence-distribution-1996@Taxes@-page-7.pdf
splitted_pdfs/pence-distribution-1996@Taxes@-page-8.pdf
splitted_pdfs/Pension-Plan-Plan-A-1 Employees@Human Resources@-page-0.pdf
splitted_pdfs/Pension-Plan-Plan-A-1 Employees@Human Resources@-page-1.pdf
splitted_pdfs/Pension-Plan-Plan-A-1 Employees@Human Resources@-page-10.pdf
splitted_pdfs/Pension-Plan-Plan-A-1 Employees@Human Resources@-page-11.pdf
splitted_pdfs/Pension-Plan-Plan-A-1 Employees@Human Resources@-page-12.pdf
splitted_pdfs/Pension-Plan-Plan-A-1 Employees@Human Resources@-page-13.pdf
splitted_pdfs/Pension-Plan-Plan-A-1 Employees@Human Resources@-page-14.pdf
splitted_pdfs/Pension-Plan-Plan-A-1 Employees@Human Resources@-page-15.pdf
splitted_pdfs/Pension-Plan-Plan-A-1 Employees@Human Resources@-pag

splitted_pdfs/plan_summary@Human Resources@-page-8.pdf
splitted_pdfs/plan_summary@Human Resources@-page-9.pdf
splitted_pdfs/Position Decription - Teacher@Human Resources@-page-0.pdf
splitted_pdfs/Position Decription - Teacher@Human Resources@-page-1.pdf
splitted_pdfs/Position Decription - Teacher@Human Resources@-page-2.pdf
splitted_pdfs/property-valuation-report@Valuations@-page-0.pdf
splitted_pdfs/property-valuation-report@Valuations@-page-1.pdf
splitted_pdfs/property-valuation-report@Valuations@-page-10.pdf
splitted_pdfs/property-valuation-report@Valuations@-page-11.pdf
splitted_pdfs/property-valuation-report@Valuations@-page-12.pdf
splitted_pdfs/property-valuation-report@Valuations@-page-13.pdf
splitted_pdfs/property-valuation-report@Valuations@-page-14.pdf
splitted_pdfs/property-valuation-report@Valuations@-page-15.pdf
splitted_pdfs/property-valuation-report@Valuations@-page-16.pdf
splitted_pdfs/property-valuation-report@Valuations@-page-17.pdf
splitted_pdfs/property-valuation-rep

splitted_pdfs/severe-weather-emergency-planning-and-preperation-checklist@Human Resources@-page-7.pdf
splitted_pdfs/severe-weather-emergency-planning-and-preperation-checklist@Human Resources@-page-8.pdf
splitted_pdfs/severe-weather-emergency-planning-and-preperation-checklist@Human Resources@-page-9.pdf
splitted_pdfs/SPD_20120101_Summary_Plan_Doc_WM@Human Resources@-page-0.pdf
splitted_pdfs/SPD_20120101_Summary_Plan_Doc_WM@Human Resources@-page-1.pdf
splitted_pdfs/SPD_20120101_Summary_Plan_Doc_WM@Human Resources@-page-10.pdf
splitted_pdfs/SPD_20120101_Summary_Plan_Doc_WM@Human Resources@-page-100.pdf
splitted_pdfs/SPD_20120101_Summary_Plan_Doc_WM@Human Resources@-page-101.pdf
splitted_pdfs/SPD_20120101_Summary_Plan_Doc_WM@Human Resources@-page-102.pdf
splitted_pdfs/SPD_20120101_Summary_Plan_Doc_WM@Human Resources@-page-103.pdf
splitted_pdfs/SPD_20120101_Summary_Plan_Doc_WM@Human Resources@-page-104.pdf
splitted_pdfs/SPD_20120101_Summary_Plan_Doc_WM@Human Resources@-page-105.pdf
splitt

splitted_pdfs/SPD_20120101_Summary_Plan_Doc_WM@Human Resources@-page-40.pdf
splitted_pdfs/SPD_20120101_Summary_Plan_Doc_WM@Human Resources@-page-41.pdf
splitted_pdfs/SPD_20120101_Summary_Plan_Doc_WM@Human Resources@-page-42.pdf
splitted_pdfs/SPD_20120101_Summary_Plan_Doc_WM@Human Resources@-page-43.pdf
splitted_pdfs/SPD_20120101_Summary_Plan_Doc_WM@Human Resources@-page-44.pdf
splitted_pdfs/SPD_20120101_Summary_Plan_Doc_WM@Human Resources@-page-45.pdf
splitted_pdfs/SPD_20120101_Summary_Plan_Doc_WM@Human Resources@-page-46.pdf
splitted_pdfs/SPD_20120101_Summary_Plan_Doc_WM@Human Resources@-page-47.pdf
splitted_pdfs/SPD_20120101_Summary_Plan_Doc_WM@Human Resources@-page-48.pdf
splitted_pdfs/SPD_20120101_Summary_Plan_Doc_WM@Human Resources@-page-49.pdf
splitted_pdfs/SPD_20120101_Summary_Plan_Doc_WM@Human Resources@-page-5.pdf
splitted_pdfs/SPD_20120101_Summary_Plan_Doc_WM@Human Resources@-page-50.pdf
splitted_pdfs/SPD_20120101_Summary_Plan_Doc_WM@Human Resources@-page-51.pdf
splitted_pdfs

splitted_pdfs/SuccessionPlanningCareerPipelineToolKit_213235_7@Human Resources@-page-6.pdf
splitted_pdfs/SuccessionPlanningCareerPipelineToolKit_213235_7@Human Resources@-page-7.pdf
splitted_pdfs/SuccessionPlanningCareerPipelineToolKit_213235_7@Human Resources@-page-8.pdf
splitted_pdfs/SuccessionPlanningCareerPipelineToolKit_213235_7@Human Resources@-page-9.pdf
splitted_pdfs/Swinney_Third_Amendment_to_Extension_of_Agreement@Agreements@-page-0.pdf
splitted_pdfs/Swinney_Third_Amendment_to_Extension_of_Agreement@Agreements@-page-1.pdf
splitted_pdfs/Swinney_Third_Amendment_to_Extension_of_Agreement@Agreements@-page-2.pdf
splitted_pdfs/Swinney_Third_Amendment_to_Extension_of_Agreement@Agreements@-page-3.pdf
splitted_pdfs/Tax Year 2016 Corporation e-File Signature Form VA-8879C@Taxes@-page-0.pdf
splitted_pdfs/Tax Year 2016 Corporation e-File Signature Form VA-8879C@Taxes@-page-1.pdf
splitted_pdfs/tax-receivable-2005-05-10@Taxes@-page-0.pdf
splitted_pdfs/tax-receivable-2005-05-10@Taxes@-page-

## Vectorizer

In [24]:
Data_Frame.tail(50)

,File_name,text,target
594,txt_folder/tax-receivable-2005-05-10@Taxes@-pa...,redemption date means the date on which the re...,Taxes
595,txt_folder/tax-receivable-2008@Taxes@-page-0.txt,tax receivable agreement this tax receivable a...,Taxes
596,txt_folder/tax-receivable-2008@Taxes@-page-1.txt,advisory firm means an accounting or law firm ...,Taxes
597,txt_folder/tax-receivable-2008@Taxes@-page-2.txt,ii the corporation consolidates with or merges...,Taxes
598,txt_folder/tax-receivable-2008@Taxes@-page-3.txt,early termination payment means as of the date...,Taxes
599,txt_folder/tax-receivable-2008@Taxes@-page-4.txt,libor means for each month or portion thereof ...,Taxes
600,txt_folder/tax-receivable-2008@Taxes@-page-5.txt,realized tax benefit means for a covered taxab...,Taxes
601,txt_folder/tax-receivable-2008@Taxes@-page-6.txt,senior obl igations means principal interest o...,Taxes
602,txt_folder/tax-receivable-2008@Taxes@-page-7.txt,by the corporation on a pro rata basis from th...,Taxes
603,txt_folder/tax-receivable-2008@Taxes@-page-8.txt,the corporation delivers or causes holdings ii...,Taxes


In [25]:
Data_Frame['text'][0]

'volunteer coordinators job description general description the volunteer manager provides professional staff support to casa volunteers ensuring that children involved with the casa program receive sound advocacy and early permanency planning the volunteer manager is responsible for volunteer supervision and coordination of cases qualifications the volunteer manager should have the following skills and experience bachelors degree in social servicerelated field or equivalent combination of education and experience the ability to communicate with supervise and empower volunteers to be effective in their rolesexperience with volunteers preferred the ability to work cooperatively with different types of personalities knowledge and understanding of issues and dynamics within families in crisis relating to child abuse and neglect given preference commitment to casa programs goals and mission accountability the volunteer manager reports directly to the executive director who is responsible f

In [26]:
"txt_folder/tax-receivable-2008_Taxes-page-2"

'txt_folder/tax-receivable-2008_Taxes-page-2'

In [27]:
Data_Frame['target'].value_counts()

Taxes              211
Human Resources    175
Valuations         119
Deeds               77
Agreements          62
Name: target, dtype: int64

In [28]:
Data_Frame.isna().sum()

File_name    0
text         0
target       0
dtype: int64

In [29]:
type(file_path.split(".")[0].split("-")[-1])

NameError: name 'file_path' is not defined

In [30]:
os.listdir("RawData _2/Deeds")

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'RawData _2/Deeds'

In [31]:
txt = '0803_job_description_vol_coordinator_0053_Human Resources_Human Resources-page-0.txt'
txt.split("-")[0].split("_")[-1]

'Human Resources'

In [12]:
text_file = "Text_1.txt"
txt = getting_text(text_file)
print(txt)

NameError: name 'getting_text' is not defined

In [13]:
for i in range(1,1):
    print(i)